# Coursera Capstone Project - Week3 - Activity 2

In [1]:
#Importing libraries 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### Importing Geospatial_Coordinates csv to dataframe
The dataframe will consist of three columns: 'Postal Code','Latitude','Longitude'

In [2]:
url = 'https://github.com/steffim88/Coursera_Capstone/blob/master/Geospatial_Coordinates.csv'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])


df_geo = pd.DataFrame(data, columns=['Index','Postal Code','Latitude','Longitude'])
df_geo = df_geo[['Postal Code','Latitude','Longitude']]
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.8066863,-79.1943534
1,M1C,43.7845351,-79.1604971
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.2169174
4,M1H,43.773136,-79.2394761


# Importing Canada csv to dataframe

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
df.dropna(inplace = True)
df = df[(df.Borough != 'Not assigned') ]
def list_neighborhood(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
df_final = df.groupby(['PostalCode', 'Borough']).apply(list_neighborhood).reset_index(name='Neighborhood')
df_final.loc[df_final['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_final['Borough']
df_final.head()



,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Join 2 dataframes

In [4]:
df_new = pd.concat([df_final,df_geo ] ,axis=1)
df_new.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",M1C,43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,M1G,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.2394761


In [5]:
df_new = df_new[ df_new.Borough.str.contains('Toronto') ]
df_new.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
37,M4E,East Toronto,The Beaches,M4E,43.6763574,-79.2930312
41,M4K,East Toronto,"Riverdale, The Danforth West",M4K,43.6795571,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",M4L,43.6689985,-79.3155716
43,M4M,East Toronto,Studio District,M4M,43.6595255,-79.340923
44,M4N,Central Toronto,Lawrence Park,M4N,43.7280205,-79.3887901


In [6]:
!pip install folium 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

In [7]:

toronto_coords = [43.653226, -79.3831843]

In [8]:
map_toronto = folium.Map(location=[toronto_coords[0], toronto_coords[1]], zoom_start=12)

In [9]:
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
  

In [10]:
  map_toronto